Mixture of Gaussians model posterior distribution estimation based on:

(1) Our MH test

(2) Korattikara's paper: Cutting the MH Budget

(3) Bardenet's paper: On Markov chain Monte Carlo methods for tall data

(4) Bardenet's paper: Towards scaling up Markov chain Monte Carlo: an adaptive subsampling approach

The mixture model is based on Section 5.1 of "Bayesian Learning via Stochastic Gradient Langevin Dynamics" (2011). 

In [16]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random
from scipy.stats import t
import sys
import scipy.io
from bisect import bisect

In [17]:
# Our MH Test
class our_mh:
    def __init__(self, x, ecdfmat, sd_vect):
        self.X = x  # data
        self.N = x.shape[0]
        self.ecdfmat = ecdfmat
        self.sd_vect = sd_vect
    
    def randomWalkProposer(self, theta, eps_sq):
        # random walk proposer to get the next parameter set
        # input: theta: 1-D Array like, of length 2
        #        eps_sq: 2-D Array like, the covariance matrix
        noise = np.random.multivariate_normal(np.array([0,0]), eps_sq).reshape((2,1))
        return theta + noise
    
    def log_f(self, theta, X, N, T):
        """
        The function 'f' is the posterior:
    
        f(theta) \propto p(\theta) * \prod_{i=1}^N p(x_i | \theta)
        """
        scale_and_temp = N / float(len(X) * T)
    
        inverse_covariance = np.array([[0.1,0],[0,1]])
        prior_constant = 1.0 / (2*np.pi*np.sqrt(10))
        prior = np.log(prior_constant) - 0.5*(theta.T).dot(inverse_covariance).dot(theta)
    
        X_all = X.reshape((len(X),1))
        ll_constant = (1.0 / (4*np.sqrt(np.pi)))
        L = ll_constant * (np.exp(-0.25*(X_all-theta[0])**2) + \
                           np.exp(-0.25*(X_all-(theta[0]+theta[1]))**2))
        log_likelihood = np.sum(np.log(L)) * scale_and_temp
    
        assert (N / float(len(X))) >= 1
        assert not np.isnan(prior + log_likelihood)
        return (prior + log_likelihood)[0,0]
    
    def get_delta(self, theta_c, theta_p, X, N, T):
        loss_old = self.log_f(theta_c, X, N, T)
        loss_new = self.log_f(theta_p, X, N, T)
        assert not np.isnan(loss_new - loss_old)
        return loss_new - loss_old

    def estimateVar_from_one_batch (self, theta_c, theta_p, X, N, T):
        n = len(X)
        # notice here, we need to set up the N and T to be 1, and rescale them later 
        tmp_delta = [self.get_delta(theta_c, theta_p, np.asarray([x]), 1, 1) for x in X]
        varRes = np.var(np.asarray(tmp_delta))
        varRes = varRes * (1.0*N/n/T)**2.0 *n
        assert not np.isnan(varRes)
        return varRes
    
    def update_delta_and_sd_one_more_minibatch(self, theta_c, theta_p, X, n, N, T, old_delta, old_sd_delta):
        newDelta = self.get_delta(theta_c, theta_p, X, N, T)
        updated_delta = (old_delta * n + newDelta)*1.0 / (n+1)
        newVar = self.estimateVar_from_one_batch(theta_c, theta_p, X, N, T)
        m = len(X)
        updated_sd = ((old_sd_delta**2.0 *(n*m)**2 + newVar* m**2) / (m*(n+1))**2)**0.5
        return (updated_delta, updated_sd)
    
    def get_rand_xcorr(self, ecdf, sdvect, estimated_sd):
        index = bisect(sdvect, estimated_sd) + 1
        x = ecdf[0,:]
        f = ecdf[index, :]
        u = np.random.random()
        return x[bisect(f, u)]
    
    def posterior_estimation(self, num_passes, \
                             rw_eps, temperature, \
                             mb_size, theta, \
                             num_samples_delta, \
                             mavg):
        our_MH_accept_1 = []
        our_MH_reject_1 = []
        our_sds_1 = []
        our_deltas_1 = []
        our_xcorrs_1 = []
        data_consume = []
        all_3 = theta
        sd = 0
        
        for T in range(1, num_passes):  
            if (T % int(num_passes/4) == 0):
                print("T={}".format(T))
            theta_new = self.randomWalkProposer(theta, rw_eps)
            X_mb_real = self.X[np.random.choice(self.N, mb_size, replace=False)]
            delta_real = self.get_delta(theta, theta_new, X_mb_real, self.N, temperature)
            
            X_mini_batch = X[np.random.choice(self.N, mb_size, replace=False)]
            this_est_std = self.estimateVar_from_one_batch(theta, theta_new, X_mini_batch, self.N, temperature)**0.5
            projected_mavg = this_est_std
            num_mini_batch = 1
            while (projected_mavg >= 1.188):
                # we need to consume more data here
                X_new_mini_batch = X[np.random.choice(self.N, mb_size, replace=False)]
                updatedval = self.update_delta_and_sd_one_more_minibatch(theta, theta_new, X_new_mini_batch,\
                                                                         num_mini_batch, self.N, temperature, delta_real,\
                                                                        projected_mavg)
                projected_mavg = updatedval[1]
                delta_real = updatedval[0]
                num_mini_batch += 1
        
            data_consume.append(num_mini_batch)
            # Now we update the moving average and take advantage of pre-computed data.
            sd = projected_mavg
            X_corr = self.get_rand_xcorr(self.ecdfmat, self.sd_vect, sd)
    
      
            # Gather data for diagnostics.
            our_deltas_1.append(delta_real)
            our_sds_1.append(sd)
            our_xcorrs_1.append(X_corr)
    
            # Now *finally* do the test!
            if (X_corr + delta_real > 0):
                theta = theta_new
                our_MH_accept_1.append(T)
            else:
                our_MH_reject_1.append(T)
            all_3 = np.concatenate((all_3,theta), axis=1)
        return (all_3, data_consume)

In [18]:
# class cut_mh:
#    def __init__(self):



In [19]:
# Generate data
N = 10000 # number of points
sigma1_sq = 10
sigma2_sq = 1
sigmax_sq = 2
theta1 = 0
theta2 = 1
X = np.zeros(N)
i = 0
while i < N :
    u = np.random.random()
    if (u<0.5):
        X[i] = np.random.normal(theta1, np.sqrt(sigmax_sq))
        i = i + 1
    elif (u>0.5):
        X[i] = np.random.normal(theta1+theta2, np.sqrt(sigmax_sq))
        i = i + 1
        
# Load XcorrCurves.mat
mat = scipy.io.loadmat('../generateXcorr/XcorrCurves.mat')
ecdfmat = mat['res']
sd_vect = mat['sdval']

In [ ]:
# *********** Run Our MH Test **********
num_passes = 1000
rw_eps = 0.03 * np.eye(2)
temperature = 110
mb_size = 200
theta = np.array([[0.5],[0]])
num_samples_delta = 5
mavg = 0.7
mhtest = our_mh(X, ecdfmat, sd_vect)

# iterate using different temperature and mb_size
for temperature in range(10,150,10):
    for mb_size in range(50,500,50):
        print 'Temperature and minibatch size are:' 
        print temperature, mb_size
        (all3, data_consume) = mhtest.posterior_estimation(num_passes, \
                             rw_eps, temperature, \
                             mb_size, theta, \
                             num_samples_delta, \
                             mavg)
        # plot result
        fig, axarr = plt.subplots(1,2,figsize=(15,5))
        axarr[0].set_title("Our Method", size="x-large")
        axarr[0].scatter(all3[0], all3[1])
        axarr[0].set_xlim([-1.5,2.5])
        axarr[0].set_ylim([-3,3])
        dataconsume = np.asarray(data_consume)
        axarr[1].set_title("Data Instances Consumed Per Iteration", size="xx-large")
        axarr[1].hist(dataconsume*200, bins=10, facecolor ='green')
        axarr[1].set_xlabel("Data Instances", size="x-large")
        axarr[1].set_ylabel("Counts", size="x-large")
        plt.tight_layout()
        plt.show()

Temperature and minibatch size are:
10 50
